In [1]:
# Import the PySpark libraries
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os 
import pyspark

os.environ["JAVA_HOME"] = "1.8.0"
os.environ['PYSPARK_SUBMIT_ARGS']="pyspark-shell"
# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Create a SQLContext
sqlContext = SQLContext(spark)

spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
data = spark.read.format("csv").option("header","true").option("inferSchema","true").load("clickstream1.csv")
data.createOrReplaceTempView("Data")
results1 = spark.sql(
  """with t1 as (SELECT 
     user_id,
     session_id,
     event_type, 
     event_page,
     row_number() over (partition by user_id, session_id, event_page order by timestamp) as rank, 
     row_number() over (order by timestamp) as number,
     timestamp
     FROM data
     --where user_id = 412
     order by timestamp), t2 as (
     select 
     user_id, 
     session_id, 
     array_agg(event_page) as array
     from t1 
     where rank = 1
     group by user_id, session_id
     order by user_id, session_id)
     
     select 
     array, 
     count(array) as count
     from t2
     group by array
     order by count desc
     limit 30
""")
results1.show(truncate=False)